In [ ]:
# ## find key phrase with TF-IDF weight
# import re
# import string
# import numpy as np
# import pandas as pd
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from itertools import chain, groupby
# from sklearn.feature_extraction.text import TfidfTransformer
# from nltk.chunk import tree2conlltags
# from nltk import word_tokenize, sent_tokenize
# from nltk.chunk.regexp import RegexpParser

In [ ]:
# rm_punct=re.compile('[{}]'.format(re.escape(string.punctuation)))
# stop_words=set(stopwords.words('english'))

# def get_cand_words(sent, cand_type='word', remove_punct=False):
#     candidates=list()
#     sent=rm_punct.sub(' ', sent)
#     tokenized=word_tokenize(sent)
#     tagged_words=pos_tag(tokenized)
#     if cand_type=='word':
#         pos_tag_patt=tags = set(['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'NNPS'])
#         tagged_words=chain.from_iterable(tagged_words)
#         for word, tag in enumerate(tagged_words):
#             if tag in pos_tag_patt and word not in stop_words:
#                 candidates.append(word)
                
# #     elif cand_type == 'phrase':
# #         grammar = r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'
# #         chunker = RegexpParser(grammar)
# #         all_tag = chain.from_iterable([chunker.parse(tag) for tag in tagged_words])
# #         for key, group in groupby(all_tag, lambda tag: tag[2] != 'O'):
# #             candidate = ' '.join([word for (word, pos, chunk) in group])
# #             if key is True and candidate not in stop_words:
# #                 candidates.append(candidate)
# #     else:
# #         print("return word or phrase as target phrase")
#     return candidates


# doc="Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran."
# get_cand_words(sent=doc, cand_type='word', remove_punct=False)

## Experiment

In [1]:
## read dataset
import pandas as pd
from nltk.corpus import stopwords
import string

dataset=pd.read_csv('train.tsv',delimiter='\t',encoding='utf-8')
dataset.columns=['ID','Label','Statement','Subject','speaker','job_title',
           'state_info','pantry_affiliation','barely_true_cnt','false_cnt',
           'half_true_cnt','mostly_true_cnt','pants_on_fire_cnt','Context']
#---------------------------------------------------#
# ## data cleaning phase
# stop=set(stopwords.words('english'))
# text_no_stops=[]
# for elm in range(0, len(dataset.index)):
#     res=' '.join([i for i in dataset['Statement'][elm].lower().split() if i not in stop])
#     text_no_stops.append(res)

# ##
# dataset['Statement']=text_no_stops
# ## remove the punctuation
# rm_punct=re.compile('[{}]'.format(re.escape(string.punctuation)))
# text_no_punc=[]
# for i in range(0, len(dataset.index)):
#     removed=rm_punct.sub(' ', dataset['Statement'][i])
#     text_no_punc.append(removed)

# dataset['Statement']=text_no_punc
# ## here is how to clean up non-letter symbols in statement columns
# all_text = []
# for i in range(0, len(dataset.index)):
#     patt = re.sub('[^a-zA-Z]', ' ', dataset['Statement'][i])
#     res = ' '.join(str(patt).lower().split())
#     all_text.append(res)

# dataset['Statement']=all_text

In [13]:
## full implementation of TF-IDF
from nltk.corpus import stopwords
import string, re

def preprocessing_txt(dataset):
    stop_words = set(stopwords.words('english'))
    rm_punct = re.compile('[{}]'.format(re.escape(string.punctuation)))
    corpus=[]
    for elm in range(0, len(dataset.index)):
        res=' '.join([i for i in dataset['Statement'][elm].lower().split() if i not in stop_words])
        res=rm_punct.sub('',res)
        corpus.append(res)
    return corpus

##


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

mydat=dataset['Statement'].tolist()
cv=CountVectorizer(max_df=0.85, stop_words=None)
word_cnt_vector=cv.fit_transform(mydat)
word_cnt_vector.shape

list(cv.vocabulary_.keys())[:10]
list(cv.get_feature_names())[2000:3000]


## TF-IDF transformation to compute IDF

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_cnt_vector)

## print idf
tfidf_transformer.idf_

##

def sort_coo(coo_matrix):
    tuples=zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

##

feature_name=cv.get_feature_names()
doc=mydat[0]
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
sorted_items=sort_coo(tf_idf_vector.tocoo())
key_words=extract_topn_from_vector(feature_name, sorted_items,10)
key_words

##
def get_keywords():
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_name,sorted_items,1)
    return keywords


In [159]:
res=[]
for i in range(0, len(mydat)):
    tf_idf_vector=tfidf_transformer.transform(cv.transform([mydat[i]]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords=extract_topn_from_vector(feature_name,sorted_items,1)
    res.append(keywords)

[word.keys() for word in res]


[dict_keys(['started']),
 dict_keys(['doubt']),
 dict_keys(['surgeries']),
 dict_keys(['turnaround']),
 dict_keys(['tenured']),
 dict_keys(['dunnam']),
 dict_keys(['watergate']),
 dict_keys(['noaa']),
 dict_keys(['leibham']),
 dict_keys(['margin']),
 dict_keys(['slipped']),
 dict_keys(['slow']),
 dict_keys(['bled']),
 dict_keys(['waived']),
 dict_keys(['chose']),
 dict_keys(['buy']),
 dict_keys(['wis']),
 dict_keys(['subsidiary']),
 dict_keys(['rico']),
 dict_keys(['adjust']),
 dict_keys(['corporate']),
 dict_keys(['auto']),
 dict_keys(['favors']),
 dict_keys(['rid']),
 dict_keys(['jonathan']),
 dict_keys(['mosques']),
 dict_keys(['landslide']),
 dict_keys(['tenn']),
 dict_keys(['youth']),
 dict_keys(['endorsing']),
 dict_keys(['climates']),
 dict_keys(['going']),
 dict_keys(['business']),
 dict_keys(['pryor']),
 dict_keys(['toilet']),
 dict_keys(['founded']),
 dict_keys(['nuclear']),
 dict_keys(['destruct']),
 dict_keys(['added']),
 dict_keys(['suggested']),
 dict_keys(['kurds']),
 di